In [ ]:
%run setup.ipynb

# Basic LLM Invocation

In [ ]:
# https://python.langchain.com/docs/integrations/chat/openai/
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    # See Choosing a Model
    model="gpt-4o-mini",
    temperature=0.9
)
name = llm.invoke("""
    I want to build a llm agent that serves as a buddy to help me organize my notes, thoughts, todos, etc. 
    Suggest a good, catchy name for this app. 
    Keep the repsonse concise and pick top 3.
""")
print(name)

# Example of Prompt Chaining

In [ ]:
from langchain.prompts import PromptTemplate

gn_pt = PromptTemplate(
    input_variables=['usecase'],
    template = """
    I want to build a llm agent that serves as {usecase}. 
    Suggest a good, catchy name for this app that creatively includes the term AI in the name but still makes sense.
    Also ensure the name for the application gives off buddy vibes.
    Keep the repsonse concise and pick top 3.
    """
)

usecase = "a buddy to help me organize my notes, thoughts, todos, etc"
gn_pt.format(usecase=usecase)

In [ ]:
gk_pt = PromptTemplate(
    input_variables=['usecase', 'names'],
    template = """
    Given I want to develop a genAI companion with one of these names: {names} for this use case: {usecase}, determine what search keywords are worth sponsoring in google search for each of the names. 
    Pick the most profitable 5 for each name.
    Format them as a JSON object where each key is the name and the value is the list is the keywords to sponsor for the name.
    """
)

usecase = "a buddy to help me organize my notes, thoughts, todos, etc"
gk_pt.format(usecase=usecase, names="a, b, c")

In [ ]:
# from langchain_core.output_parsers.json import SimpleJsonOutputParser
from IPython.display import display
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser

from operator import itemgetter

name_gen_chain = gn_pt | llm | StrOutputParser()

# https://python.langchain.com/docs/how_to/output_parser_json/ 
keyword_gen_chain = gk_pt | llm | JsonOutputParser()

# LCEL: https://stackoverflow.com/questions/76788293/how-to-include-the-inputs-of-the-first-chain-to-the-second-chain-in-langchains
# Lang Chain Expression Language
overall_chain = {"names": name_gen_chain, "usecase": itemgetter("usecase")} | keyword_gen_chain

In [ ]:
import json
print(json.dumps(overall_chain.invoke({'usecase': usecase}), indent=4))